# 3.1 The transforms module

## 3.1.1 Single-tape and quantum function transforms

The following single tape transform implements the circuit identity 

$$
CNOT_{ij} = H_j \cdot CZ_{ij} \cdot H_j,
$$ 

and applies it directly to a quantum tape.

In [1]:
import pennylane as qml

@qml.single_tape_transform
def convert_cnots(tape):
    for op in tape:
        if op.name == 'CNOT':
            qml.Hadamard(wires=op.wires[1])
            qml.CZ(wires=[op.wires[0], op.wires[1]])
            qml.Hadamard(wires=op.wires[1])
        else: 
            qml.apply(op)

with qml.tape.QuantumTape() as tape:
    qml.CNOT(wires=[0, 1])
    qml.RX(0.1, wires=0)
    qml.CNOT(wires=[1, 2])
    qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

transformed_tape = convert_cnots(tape)

In [2]:
print(qml.drawer.tape_text(tape))

0: ─╭●──RX─┤ ╭<Z@Z>
1: ─╰X─╭●──┤ ╰<Z@Z>
2: ────╰X──┤       


In [3]:
print(qml.drawer.tape_text(transformed_tape))

1: ──H─╭Z──H──╭●────┤ ╭<Z@Z>
0: ────╰●──RX─│─────┤ ╰<Z@Z>
2: ──H────────╰Z──H─┤       


A tape transform can be elevated to a quantum function transform (qfunc transform) by swapping out the top-level decorator.

In [4]:
@qml.qfunc_transform
def convert_cnots(tape):
    for op in tape:
        if op.name == 'CNOT':
            qml.Hadamard(wires=op.wires[1])
            qml.CZ(wires=[op.wires[0], op.wires[1]])
            qml.Hadamard(wires=op.wires[1])
        else:
            qml.apply(op)

Once defined, qfunc transforms can be applied to quantum functions as a decorator. This enables us to easily compose qfunc transforms by stacking decorators.

In [5]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
@convert_cnots
def circuit(param):
    qml.CNOT(wires=[0, 1])
    qml.RX(param, wires=0)
    qml.CNOT(wires=[1, 2])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [6]:
circuit(0.5)

tensor(0.87758256, requires_grad=True)

In [7]:
print(qml.draw(circuit)(0.3))

0: ────╭●──RX(0.30)───────┤ ╭<Z@Z>
1: ──H─╰Z──H────────╭●────┤ ╰<Z@Z>
2: ──H──────────────╰Z──H─┤       


We can apply classical processing to a tape's parameters in a way that preserves their differentiability. This can be done in a framework-agnostic way using the `qml.math` module.

In [8]:
import pennylane.math as math

@qml.qfunc_transform
def square_root_rx(tape):
    for op in tape:
        if op.name == 'RX':
            # Modify each RX gate to apply sqrt of rotation angle
            qml.RX(math.sqrt(op.data[0]), wires=op.wires[0])
        else :
            qml.apply(op)

Below is an example of this using the PyTorch framework.

In [9]:
import torch

def apply_rx(x):
    qml.RX(x, wires=0)
    return qml.expval(qml.PauliZ(0))

dev = qml.device('default.qubit', wires=1)

# Create a QNode with the untransformed function and compute the gradient
qnode = qml.QNode(apply_rx, dev, interface='torch')

x_orig = torch.tensor(0.3, requires_grad=True)
res = qnode(x_orig)
res.backward()

# Create a QNode with the transformed function and again compute the gradient
transformed_qnode = qml.QNode(square_root_rx(apply_rx), dev, interface='torch')

x_transformed = torch.tensor(0.3, requires_grad=True)
res = transformed_qnode(x_transformed)
res.backward()

In [10]:
print(x_orig.grad)

tensor(-0.2955)


In [11]:
print(x_transformed.grad)

tensor(-0.4754)


## 3.1.2 Batch transforms

We construct a Hamiltonian and tape as per Figure 4, taking $c_1 = c_2 = c_3 = 1$.

In [12]:
# PennyLane has a built-in NumPy that has been adjusted to work with
# automatic differentation.
from pennylane import numpy as np

coeffs = np.array([1, 1, 1])
obs = [
    qml.PauliZ(0) @ qml.PauliZ(1), 
    qml.PauliY(0) @ qml.PauliY(1), 
    qml.PauliX(0) @ qml.PauliX(1)
]

H = qml.Hamiltonian(coeffs, obs)

with qml.tape.QuantumTape() as tape:
    qml.RY(0.3, wires=0)
    qml.RY(0.4, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 0])
    qml.expval(H)

Applying the `hamiltonian_expand` transform gives us back a list of tapes, and a processing function.

In [13]:
tapes, fn = qml.transforms.hamiltonian_expand(tape)

In [14]:
for t in tapes:
    print(qml.drawer.tape_text(t), end="\n\n")

0: ──RY─╭●─╭X─┤ ╭<Z@Z>
1: ──RY─╰X─╰●─┤ ╰<Z@Z>

0: ──RY─╭●─╭X─┤ ╭<Y@Y>
1: ──RY─╰X─╰●─┤ ╰<Y@Y>

0: ──RY─╭●─╭X─┤ ╭<X@X>
1: ──RY─╰X─╰●─┤ ╰<X@X>



Note that we have one tape per observable. Let's now create a device, and execute the tapes.

In [15]:
dev = qml.device('default.qubit', wires=2)
results = qml.execute(tapes, dev, gradient_fn=None)

In [16]:
print(res)

tensor(0.8537, dtype=torch.float64, grad_fn=<SqueezeBackward0>)


The processing function `fn` can now be applied to these results.

In [17]:
fn(results)

array(0.97272928)

## 3.1.3 Other transforms

Two other types of transforms exist in PennyLane: device transforms, and information transforms. An example below is show for a device transform that applies amplitude damping after every gate.

In [18]:
def circuit(x):
    qml.RX(-2*x, wires=0)
    qml.S(wires=0)    
    return qml.probs(wires=0)

dev = qml.device('default.mixed', wires=1)
qnode = qml.QNode(circuit, dev)

# Adds amplitude damping after every gate at the device level
noisy_dev = qml.transforms.insert(qml.AmplitudeDamping, 0.05, position="all")(dev)

# Create a QNode on the transformed device
noisy_qnode = qml.QNode(circuit, noisy_dev)

An example of an information transform is `qml.draw`. It takes a QNode as input and return a function that draws it.

In [19]:
print(qml.draw(qnode, expansion_strategy="device")(0.3))

0: ──RX(-0.60)──S─┤  Probs


In [20]:
print(qml.draw(noisy_qnode, expansion_strategy="device")(0.3))

0: ──RX(-0.60)──AmplitudeDamping(0.05)──S──AmplitudeDamping(0.05)─┤  Probs
